**Mount Drive**

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


# Prepocess Data

Convert Data to Yolo format for Yolov7 Training

In [3]:
!pip install pylabel -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 367.8/367.8 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 33.2 MB/s eta 0:00:00


In [4]:
from pylabel import importer

In [5]:
annot_folder = "/content/drive/MyDrive/license_plate_det/plate_data/annotations"
image_folder = "/content/drive/MyDrive/license_plate_det/plate_data/images"

In [6]:
dataset = importer.ImportVOC(path=annot_folder, path_to_images=image_folder, name="plate_dataset")
dataset.df.head(5)

Importing VOC files...: 100%|██████████| 434/434 [00:07<00:00, 56.15it/s] 


,img_folder,img_filename,img_path,img_id,img_width,img_height,img_depth,ann_segmented,ann_bbox_xmin,ann_bbox_ymin,...,ann_iscrowd,ann_keypoints,ann_pose,ann_truncated,ann_difficult,cat_id,cat_name,cat_supercategory,split,annotated
id,,,,,,,,,,,,,,,,,,,,,
0,/content/drive/MyDrive/license_plate_det/plate...,Cars119.png,NaN,0,400,247,3,0,187.0,140.0,...,NaN,NaN,Unspecified,0,0,0,licence,NaN,,1
1,/content/drive/MyDrive/license_plate_det/plate...,Cars103.png,NaN,1,400,196,3,0,230.0,129.0,...,NaN,NaN,Unspecified,0,0,0,licence,NaN,,1
2,/content/drive/MyDrive/license_plate_det/plate...,Cars103.png,NaN,1,400,196,3,0,189.0,116.0,...,NaN,NaN,Unspecified,0,0,0,licence,NaN,,1
3,/content/drive/MyDrive/license_plate_det/plate...,Cars133.png,NaN,2,600,411,3,0,247.0,185.0,...,NaN,NaN,Unspecified,0,0,0,licence,NaN,,1
4,/content/drive/MyDrive/license_plate_det/plate...,Cars117.png,NaN,3,400,185,3,0,42.0,126.0,...,NaN,NaN,Unspecified,0,0,0,licence,NaN,,1


In [7]:
print(f"Number of images: {dataset.analyze.num_images}")
print(f"Number of classes: {dataset.analyze.num_classes}")
print(f"Classes:{dataset.analyze.classes}")
print(f"Class counts:\n{dataset.analyze.class_counts}")

Number of images: 433
Number of classes: 1
Classes:['licence']
Class counts:
cat_name
licence    471
Name: count, dtype: int64


In [8]:
# split dataset into train, val, test [6%:3.5%:.5%]
dataset.splitter.GroupShuffleSplit(train_pct=.6, val_pct=.35, test_pct=0.05)
dataset.analyze.ShowClassSplits()

,all,train,test,val
cat_name,,,,


In [9]:
# create coco annotations
dataset.export.ExportToCoco()

Exporting to COCO file...: 100%|██████████| 471/471 [00:00<00:00, 1040.82it/s]


['/content/drive/MyDrive/license_plate_det/plate_data/annotations/plate_dataset.json']

In [10]:
# create yolo format dataset
output_path = "/content/drive/MyDrive/license_plate_det/plate_coco/labels"
dataset.export.ExportToYoloV5(output_path=output_path, yaml_file='dataset.yaml', copy_images=True, use_splits=True)

Exporting YOLO files...: 100%|██████████| 433/433 [01:52<00:00,  3.86it/s]


['/content/drive/MyDrive/license_plate_det/plate_coco/dataset.yaml',
 '/content/drive/MyDrive/license_plate_det/plate_coco/labels/train/Cars119.txt',
 '/content/drive/MyDrive/license_plate_det/plate_coco/labels/train/Cars103.txt',
 '/content/drive/MyDrive/license_plate_det/plate_coco/labels/train/Cars133.txt',
 '/content/drive/MyDrive/license_plate_det/plate_coco/labels/train/Cars13.txt',
 '/content/drive/MyDrive/license_plate_det/plate_coco/labels/train/Cars112.txt',
 '/content/drive/MyDrive/license_plate_det/plate_coco/labels/train/Cars132.txt',
 '/content/drive/MyDrive/license_plate_det/plate_coco/labels/train/Cars113.txt',
 '/content/drive/MyDrive/license_plate_det/plate_coco/labels/train/Cars124.txt',
 '/content/drive/MyDrive/license_plate_det/plate_coco/labels/train/Cars12.txt',
 '/content/drive/MyDrive/license_plate_det/plate_coco/labels/train/Cars129.txt',
 '/content/drive/MyDrive/license_plate_det/plate_coco/labels/train/Cars106.txt',
 '/content/drive/MyDrive/license_plate_det

In [11]:
import os

print(f"Train data: {len(os.listdir('/content/drive/MyDrive/license_plate_det/plate_coco/images/train'))}")
print(f"Val data: {len(os.listdir('/content/drive/MyDrive/license_plate_det/plate_coco/images/val'))}")
print(f"Test data: {len(os.listdir('/content/drive/MyDrive/license_plate_det/plate_coco/images/test'))}")

Train data: 265
Val data: 147
Test data: 21


In [12]:
# copy coco annotation to yolo dataset folder
os.makedirs("/content/drive/MyDrive/license_plate_det/plate_coco/annotations", exist_ok=True)

!cp /content/drive/MyDrive/license_plate_det/plate_data/annotations/plate_dataset.json \
  /content/drive/MyDrive/license_plate_det/plate_coco/annotations